# Bias de post-tratamiento

> Bias: a systematic distortion of a statistical result due to a factor not allowed for in its derivation.

Es común preocuparse por inferencias erróneas producidas por omitir los predictores suficientes. Este tipo de errores se les llama **bias de omisión de variables**, y lo que vimos en el tema anterior nos da una muestra de lo que puede suceder.

Sin embargo, no es común preocuparse por inferencias erróneas por incluir predictores. Sin embargo, el **bias de inclusión de variables** es real. Meter todas las variables en nuestro modelo no es una buena idea.

Este tipo de bias se puede presentar en varias formas. La primera que veremos es el **bias de post-tratamiento**.

> **Objetivos:**
> - Entender el problema de bias de post-tratamiento.


> **Referencias:**
> 
> - Statistical Rethinking: A Bayesian Course with Examples in R and Stan (2nd edition) - Richard McElreath.

## 1. Bias de post-tratamiento

La palabra post-tratamiento proviene de la idea de diseño de experimentos.

Supongamos que cultivamos plantas en un invernadero, y queremos saber la diferencia entre cultivarlas bajo los efectos de diferentes fungicidas, dado que los hongos en las plantas suelen reducir su crecimiento.

Inicialmente las plantas son sembradas, y luego brotan, momento en el cual sus tamaños son medidos. Los diferentes tratamientos son llevados a cabo. Después, se miden sus tamaños finales así como la presencia de hongos.

Nuestras variables son entonces:

* Altura inicial
* Altura final -> Resultado
* Tratamiento
* Presencia de hongos

¿Cuáles de estas variables deberíamos incluir en un modelo para predecir la altura final de las plantas? Sucede que si queremos razonar de manera causal, no deberíamos incluir la variable de presencia de hongos, dado que es un efecto post-tratamiento.

Veamos esto con datos simulados:

In [ ]:
# Importar scipy.stats norm, binom

# Importar numpy

# Importar pandas


In [ ]:
# Número de plantas


# Alturas iniciales


# Simulamos tratamiento para la mitad de las plantas

# Simulamos presencia de hongos

# Simulamos altura final


# Dataframe


In [ ]:
# método head


Al diseñar el modelo, debemos hacer de cuenta que no conocemos el proceso que genera los datos. En una situación real no conoceremos dicho proceso, más tendremos información científica que nos apoyará en la construcción del modelo.

En este caso, sabemos que las plantas al final serán más altas que al inicio. De esta manera, si modelamos los parámetros de forma proporcional al tamaño de las plantas al inicio, será más fácil razonar acerca de las previas. Enfocándonos en las variables de tamaño:

$$
\begin{array}{ccc}
h_{end, i} & \sim & \text{Normal}(\mu_i, \sigma) \\
\mu_i & = & h_{ini, i} \times p
\end{array}
$$

En este caso, $p$ es la proporción promedio del tamaño final respecto al tamaño inicial:

- Si $p=1$, quiere decir que la planta no creció.
- Si $p=2$, quiere decir que la planta duplicó su tamaño.

De forma que debemos centrar nuestra previa de $p$ en 1, e incluso dejar que $p$ pueda ser menor a 1 (recordemos que no sabemos como se generaron los datos). Sin embargo, debemos asegurar que $p$ sea positiva. Una buena elección es:

$$
p \sim \text{Log-Normal}(0, 0.25)
$$

In [ ]:
# Importar scipy.stats lognorm

# Importar pyplot


In [ ]:
# Densidad log-normal


Por lo que esta elección espera una reducción desde el 40% hasta un aumente del 50%.

Ajustemos este modelo inicial:

In [ ]:
# Importar pymc

# Importar arviz


In [ ]:
# Modelo de altura de plantas


In [ ]:
# Summary


Se espera un crecimiento promedio del 40%.

Ahora, incluyamos las variables de tratamiento y hongos. Recordemos que los parámetros estarán en la escala de la proporción:

$$
\begin{array}{ccc}
h_{end, i} & \sim & \text{Normal}(\mu_i, \sigma) \\
\mu_i & = & h_{ini, i} \times p \\
p & = & \alpha + \beta_T t_i + \beta_F f_i \\
\alpha & \sim & \text{Log-Normal}(0, 0.25) \\
\beta_T & \sim & \text{Normal}(0, 0.5) \\
\beta_F & \sim & \text{Normal}(0, 0.5) \\
\sigma  & \sim & \text{Exponential}(1) \\
\end{array}
$$

con esta elección, esperamos con 95% de certeza que que la proporción se reducirá -1 (-100%) o se incrementará +1 (+100%):

In [ ]:
# Modelo predictivo


In [ ]:
# Summary


El parámetro $\alpha$ es idéntico al que estimamos antes para $p$. La posterior para $b_T$, que representa el efecto del tratamiento es prácticamente cero, lo que nos dice que el tratamiento no está asociado con el creciemiento, lo cual sabemos no es cierto (nosotros construimos los datos). **¿Qué pasó?**

El problema es que la variables indicadora de hongos es una consecuencia directa del tratamiento. Es decir, es una **variable post-tratamiento**. De forma que cuando controlamos (incluimos la variable en el modelo) para los hongos, el modelo está respondiendo la pregunta:

> Una vez sabemos que la planta desarrolló o no hongos, ¿Tiene algún efecto el tratamiento en el crecimiento?

La respuesta es un rotundo **no**, pues el tratamiento impacta el crecimiento a través de reducir los hongos. ¿Qué debemos hacer?

$$
\begin{array}{ccc}
h_{end, i} & \sim & \text{Normal}(\mu_i, \sigma) \\
\mu_i & = & h_{ini, i} \times p \\
p & = & \alpha + \beta_T t_i \\
\alpha & \sim & \text{Log-Normal}(0, 0.25) \\
\beta_T & \sim & \text{Normal}(0, 0.5) \\
\sigma  & \sim & \text{Exponential}(1) \\
\end{array}
$$

In [ ]:
# Modelo de solo el tratamiento


In [ ]:
# Summary


Ahora podemos observar el efecto positivo del tratamiento en el crecimiento de las plantas, como debe de ser. Hace sentido controlar las variables *pre-tratamiento*, como el tamaño inicial, sin embargo, las variables post-tratamiento enmascaran el efecto del tratamiento en sí.

La pregunta es entonces, ¿porqué sucede esto?

Para esto, veamos cuál es el flujo de influencia probabilística a través de un modelo gráfico:

In [ ]:
# Importar daft
import daft
# Importar causalgraphicalmodels
from causalgraphicalmodels import CausalGraphicalModel

In [ ]:
# Función para dibujar modelos gráficos
def draw_dag(dag, coordinates):
    pgm = daft.PGM()
    for node in dag.dag.nodes:
        pgm.add_node(node, node, *coordinates[node])
    for edge in dag.dag.edges:
        pgm.add_edge(*edge)
    pgm.render()
    plt.gca().invert_yaxis()

In [ ]:
plant_dag = CausalGraphicalModel(
    nodes=["Hini", "Hend", "F", "T"],
    edges=[("Hini", "Hend"), ("F", "Hend"), ("T", "F")]
)
coordinates = {"Hini": (0, 0), "T": (4, 0), "F": (3, 0), "Hend": (2, 0)}

draw_dag(plant_dag, coordinates)

El tratamiento $T$ influencia la presencia de hongos $F$, la cual a su vez influencia el tamaño final de la planta $H_{end}$. El tamaño final de la planta también es influenciado por el tamaño inicial de laplanta $H_{ini}$.

Lo que sucede cuando incluimos la variable de hongos en el modelo es que **bloqueamos** el flujo de influencia probabilística de $T$ hacia $H_{end}$.

In [ ]:
plant_dag.get_all_independence_relationships()